Claim Severity Prediction (Risk Model)

In [2]:
import pandas as pd
df=pd.read_csv('../Data/MachineLearningRating_v3.txt',sep='|')

C:\Users\Hp\AppData\Local\Temp\ipykernel_17792\4287355953.py:2: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('../Data/MachineLearningRating_v3.txt',sep='|')


In [3]:
df['RegistrationYear'].unique()

array([2004, 2006, 2009, 2011, 2007, 2014, 2010, 2013, 2008, 2015, 2012,
       2005, 1998, 1995, 2000, 2003, 1999, 2001, 1997, 1994, 2002, 1996,
       1992, 1987, 1988])

feature engennering 

In [27]:
claim_df=df[df['TotalClaims'] >0].copy()
df['Vehicle_Age']=2023-df['RegistrationYear']
 

#feature exteraction 
x=claim_df[[ 'RegistrationYear', 'Province', 'NewVehicle', 'Vehicle_Age','Gender']]

y=claim_df['TotalClaims']



In [46]:
categorical_cols=x.select_dtypes(include=['object','category']).columns.to_list()
numerical_cols=x.select_dtypes(include=['float64','int64']).columns.to_list()
print(numerical_cols)
numerical_df=x[['RegistrationYear', 'Vehicle_Age']]

['RegistrationYear', 'Vehicle_Age']


handle missing data

In [29]:

for col in x.columns:
    if x[col].dtype=='object':   
        most_common= x[col].mode()[0]   
        x[col].fillna(most_common, inplace=True)
    else:  # Numerical columns
        median_val = x[col].median()
        x[col].fillna(median_val, inplace=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_17792\545793836.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x[col].fillna(median_val, inplace=True)
C:\Users\Hp\AppData\Local\Temp\ipykernel_17792\545793836.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col].fillna(median_val, inplace=True)
C:\Users\Hp\AppData\Local\Temp\ipykernel_17792\

handle missing value

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder=OneHotEncoder(handle_unknown='ignore')
encoded_cols=encoder.fit(x[categorical_cols])
feature_names=encoded_cols.get_feature_names_out(categorical_cols)


['Province_Eastern Cape' 'Province_Free State' 'Province_Gauteng'
 'Province_KwaZulu-Natal' 'Province_Limpopo' 'Province_Mpumalanga'
 'Province_North West' 'Province_Northern Cape' 'Province_Western Cape'
 'NewVehicle_More than 6 months' 'Gender_Female' 'Gender_Male'
 'Gender_Not specified']


In [41]:

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_cols = encoder.fit_transform(x[categorical_cols])
encoded_df=pd.DataFrame(encoded_cols,columns=feature_names)

In [ ]:
final_x=pd.concat([encoded_df,numerical_df],axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train_sev,x_test_sev,y_train_sev,y_test_sev=train_test_split(test_size=0.2,random_state=42)

Claim Probablity

In [80]:
x=df[ ['RegistrationYear', 'Province', 'NewVehicle', 'Vehicle_Age','Gender']]
y=df['Has_Claim']
categorical_cols=x.select_dtypes(exclude=['float64','int64'])
numerical_cols=x.select_dtypes(include=['float64','int64'])
for c in categorical_cols:
    x[c]=x[c].fillna(x[c].mode())
for c in numerical_cols:
    x[c]=x[c].fillna(x[c].median())
print(categorical_cols.dtypes)

C:\Users\Hp\AppData\Local\Temp\ipykernel_17792\1365624263.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[c]=x[c].fillna(x[c].mode())


Province      object
NewVehicle    object
Gender        object
dtype: object


C:\Users\Hp\AppData\Local\Temp\ipykernel_17792\1365624263.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[c]=x[c].fillna(x[c].median())
C:\Users\Hp\AppData\Local\Temp\ipykernel_17792\1365624263.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[c]=x[c].fillna(x[c].median())


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBRegressor, XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
# import shap

# Load data
df = pd.read_csv('../Data/MachineLearningRating_v3.txt', sep='|')

# Feature Engineering - create copies to avoid chained assignment
df = df.copy()
df['Has_Claim'] = (df['TotalClaims'] > 0).astype(int)
df['Vehicle_Age'] = 2023 - df['RegistrationYear']

# 1. CLAIM SEVERITY MODEL - Only policies with claims
claim_df = df[df['TotalClaims'] > 0].copy()

# Feature selection
features_severity = ['VehicleType', 'Vehicle_Age', 'Province', 'cubiccapacity', 'SumInsured']
target_severity = 'TotalClaims'

# Create a copy for processing
X_sev = claim_df[features_severity].copy()
y_sev = claim_df[target_severity].copy()

# Handle missing values without chained assignment
for col in X_sev.columns:
    if X_sev[col].dtype == 'object':
        fill_val = X_sev[col].mode()[0] if not X_sev[col].mode().empty else 'Missing'
        X_sev.loc[:, col] = X_sev[col].fillna(fill_val)
    else:
        fill_val = X_sev[col].median()
        X_sev.loc[:, col] = X_sev[col].fillna(fill_val)

# One-Hot Encoding
X_sev_encoded = pd.get_dummies(X_sev, columns=['VehicleType', 'Province'], drop_first=True)

# Train-test split
X_train_sev, X_test_sev, y_train_sev, y_test_sev = train_test_split(
    X_sev_encoded, y_sev, test_size=0.2, random_state=42
)

# 2. CLAIM PROBABILITY MODEL - All policies
features_prob = ['Vehicle_Age', 'Gender', 'Province', 'SumInsured', 'VehicleType']
target_prob = 'Has_Claim'

# Create a copy for processing
X_prob = df[features_prob].copy()
y_prob = df[target_prob].copy()

# Handle missing values without chained assignment
for col in X_prob.columns:
    if X_prob[col].dtype == 'object':
        fill_val = X_prob[col].mode()[0] if not X_prob[col].mode().empty else 'Missing'
        X_prob.loc[:, col] = X_prob[col].fillna(fill_val)
    else:
        fill_val = X_prob[col].median()
        X_prob.loc[:, col] = X_prob[col].fillna(fill_val)

# One-Hot Encoding
X_prob_encoded = pd.get_dummies(X_prob, columns=['Gender', 'Province', 'VehicleType'], drop_first=True)

# Train-test split
X_train_prob, X_test_prob, y_train_prob, y_test_prob = train_test_split(
    X_prob_encoded, y_prob, test_size=0.2, random_state=42
)

# =====================================================================
# MODEL BUILDING AND EVALUATION - REGRESSION (Claim Severity)
# =====================================================================
def train_evaluate_regression(models, X_train, X_test, y_train, y_test):
    results = {}
    for name, model in models.items():
        # Train model
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        # Store results
        results[name] = {
            'RMSE': rmse,
            'R-squared': r2,
            'model': model
        }
        
        print(f"{name}:")
        print(f"  RMSE: {rmse:.2f}")
        print(f"  R-squared: {r2:.4f}")
        print("-" * 50)
    
    return results

# Regression models
reg_models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree': DecisionTreeRegressor(max_depth=5, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
}

print("=" * 50)
print("CLAIM SEVERITY MODELS (REGRESSION)")
print("=" * 50)
severity_results = train_evaluate_regression(reg_models, X_train_sev, X_test_sev, y_train_sev, y_test_sev)

# =====================================================================
# MODEL BUILDING AND EVALUATION - CLASSIFICATION (Claim Probability)
# =====================================================================
def train_evaluate_classification(models, X_train, X_test, y_train, y_test):
    results = {}
    for name, model in models.items():
        # Train model
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)
        
        # Calculate metrics
        acc = accuracy_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Store results
        results[name] = {
            'Accuracy': acc,
            'Precision': prec,
            'Recall': rec,
            'F1': f1,
            'model': model
        }
        
        print(f"{name}:")
        print(f"  Accuracy: {acc:.4f}")
        print(f"  Precision: {prec:.4f}")
        print(f"  Recall: {rec:.4f}")
        print(f"  F1 Score: {f1:.4f}")
        print("-" * 50)
    
    return results

# Classification models
clf_models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
}

print("\n" + "=" * 50)
print("CLAIM PROBABILITY MODELS (CLASSIFICATION)")
print("=" * 50)
prob_results = train_evaluate_classification(clf_models, X_train_prob, X_test_prob, y_train_prob, y_test_prob)

# =====================================================================
# FEATURE IMPORTANCE ANALYSIS
# =====================================================================
print("\n" + "=" * 50)
print("FEATURE IMPORTANCE ANALYSIS")
print("=" * 50)

# 1. For Severity Model (Random Forest)
best_sev_model = severity_results['Random Forest']['model']
importances = best_sev_model.feature_importances_
feature_names = X_train_sev.columns

# Create DataFrame for visualization
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values('Importance', ascending=False).head(10)

print("\nTop 10 Features for Claim Severity (Random Forest):")
print(importance_df)

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_df)
plt.title('Top 10 Features - Claim Severity Prediction')
plt.tight_layout()
plt.savefig('severity_feature_importance.png')
plt.show()

# 2. For Probability Model (XGBoost)
best_prob_model = prob_results['XGBoost']['model']
importances_prob = best_prob_model.feature_importances_
feature_names_prob = X_train_prob.columns

# Create DataFrame for visualization
importance_prob_df = pd.DataFrame({
    'Feature': feature_names_prob,
    'Importance': importances_prob
}).sort_values('Importance', ascending=False).head(10)

print("\nTop 10 Features for Claim Probability (XGBoost):")
print(importance_prob_df)

# Plot feature importance
plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', data=importance_prob_df)
plt.title('Top 10 Features - Claim Probability Prediction')
plt.tight_layout()
plt.savefig('probability_feature_importance.png')
plt.show()

# =====================================================================
# SHAP ANALYSIS
# =====================================================================
print("\n" + "=" * 50)
print("SHAP ANALYSIS")
print("=" * 50)

# 1. For Severity Model
print("\nSHAP Analysis for Claim Severity (Random Forest):")
explainer_sev = shap.TreeExplainer(best_sev_model)
shap_values_sev = explainer_sev.shap_values(X_test_sev)

# Summary plot
shap.summary_plot(shap_values_sev, X_test_sev, feature_names=feature_names.tolist(), show=False)
plt.title('SHAP Summary - Claim Severity')
plt.tight_layout()
plt.savefig('shap_severity.png')
plt.show()

# 2. For Probability Model
print("\nSHAP Analysis for Claim Probability (XGBoost):")
explainer_prob = shap.TreeExplainer(best_prob_model)
shap_values_prob = explainer_prob.shap_values(X_test_prob)

# Summary plot
shap.summary_plot(shap_values_prob, X_test_prob, feature_names=feature_names_prob.tolist(), show=False)
plt.title('SHAP Summary - Claim Probability')
plt.tight_layout()
plt.savefig('shap_probability.png')
plt.show()

# =====================================================================
# MODEL COMPARISON REPORT
# =====================================================================
print("\n" + "=" * 50)
print("MODEL COMPARISON REPORT")
print("=" * 50)

# Regression model comparison
print("\nRegression Models Performance (Claim Severity):")
reg_comparison = pd.DataFrame({
    model: {metric: values[metric] for metric in ['RMSE', 'R-squared']}
    for model, values in severity_results.items()
}).T
print(reg_comparison)

# Classification model comparison
print("\nClassification Models Performance (Claim Probability):")
clf_comparison = pd.DataFrame({
    model: {metric: values[metric] for metric in ['Accuracy', 'Precision', 'Recall', 'F1']}
    for model, values in prob_results.items()
}).T
print(clf_comparison)

# Save results to CSV
reg_comparison.to_csv('regression_model_comparison.csv')
clf_comparison.to_csv('classification_model_comparison.csv')

C:\Users\Hp\AppData\Local\Temp\ipykernel_17792\3753894760.py:15: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Data/MachineLearningRating_v3.txt', sep='|')


CLAIM SEVERITY MODELS (REGRESSION)
Linear Regression:
  RMSE: 38702.90
  R-squared: 0.0686
--------------------------------------------------
Decision Tree:
  RMSE: 33400.17
  R-squared: 0.3063
--------------------------------------------------
Random Forest:
  RMSE: 36493.05
  R-squared: 0.1719
--------------------------------------------------
XGBoost:
  RMSE: 38190.34
  R-squared: 0.0931
--------------------------------------------------

CLAIM PROBABILITY MODELS (CLASSIFICATION)


c:\Users\Hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Logistic Regression:
  Accuracy: 0.9971
  Precision: 0.0000
  Recall: 0.0000
  F1 Score: 0.0000
--------------------------------------------------


c:\Users\Hp\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Decision Tree:
  Accuracy: 0.9971
  Precision: 0.0000
  Recall: 0.0000
  F1 Score: 0.0000
--------------------------------------------------


In [84]:
!pip install shap

   ---------------------------------------- 0.0/545.1 kB ? eta -:--:--
   ------------------- -------------------- 262.1/545.1 kB ? eta -:--:--
   ---------------------------------------- 545.1/545.1 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.8 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.8 MB ? eta -:--:--
   ------- -------------------------------- 0.5/2.8 MB 1.6 MB/s eta 0:00:02
   ----------- ---------------------------- 0.8/2.8 MB 1.5 MB/s eta 0:00:02
   -------------- ------------------------- 1.0/2.8 MB 1.4 MB/s eta 0:00:02
   ------------------ --------------------- 1.3/2.8 MB 1.5 MB/s eta 0:00:02
   ------------------------- -------------- 1.8/2.8 MB 1.5 MB/s eta 0:00:01
   ----------------------------- ---------- 2.1/2.8 MB 1.5 MB/s eta 0:00:01
   --------------------------------- ------ 2.4/2.8 MB 1.5 MB/s eta 0:00:01
   ------------------------------------- -- 2.6/2.8 MB 1.5 MB/s eta 0:00:01
   ---------------------------

  You can safely remove it manually.
  You can safely remove it manually.
